# Location Dataframe

An inital dataframe with city names, longitude, latitudes and distances between each city.

## 1. Import Packages

In [2]:
# import standard packages
import numpy as np
import pandas as pandas

# import visualization packages
import plotly.express as px

## 2. Create Dataframe

In [8]:
# Create column names
columns = ['city', 'long', 'lat']

# Create a dataframe
df = pandas.DataFrame(columns=columns)

,city,long,lat


In [9]:
# Save list for cities
cities = ['Salt Lake City', 'Boston', 'Phoenix', 'Seattle', 'New York City', 'Los Angeles', 'Chicago', 'Long Branch']

# Input cities into city column
df['city'] = cities

In [12]:
# Use API to obtain latitude and longitude for each city